# Tutorial 5: Prefill

In this tutorial, we will explore how to use prefils in RadPrompter.


Prefils in LLMS (Language Model-based Medical Systems) refer to pre-filled information or context that is provided to the model before generating responses. These prefils can be used to guide the model's behavior and ensure that it generates more accurate and relevant responses.

By providing prefils, LLMS can be customized to generate responses that align with specific requirements or scenarios. Prefils help in fine-tuning the model's behavior and improving the quality of generated responses.

## 0. Imports

In [3]:
import os
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient

## 1. Load reports

In [4]:
report_files = glob.glob("../../sample_reports/*.txt")

In [5]:
reports = []
report_ids = []
for report_file in report_files:
    report_ids.append(os.path.basename(report_file))
    with open(report_file, 'r') as f:
        reports.append(f.read())

# 2. Initializing Prompt

Here we load `5_Assistant-Prefill.toml`

The section to be noted is the following:

```toml
assistant_response_template = """
<json>
{
  "{{variable_name}}" : \""""
```

This prefill requires the model to generate the outputs with that json syntax.

We also add a stop tag in the `CONSTRUCTOR` section:

```toml
[CONSTRUCTOR]
system = "rdp(system_prompt)"
user = "rdp(user_prompt_intro + user_prompt_cot)"
stop_tags = "</json>"
```

This will tell the LLM to stop generating response once it reaches those tags.

In [6]:
prompt = Prompt('5_Assistant-Prefill.toml')

In [7]:
prompt

In [8]:
prompt.schemas[0]

In [9]:
prompt.schemas[1]

## 3. Initializing LLM Client and RadPrompter Engine

In [10]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

In [11]:
engine = RadPrompter(
    client=client,
    prompt=prompt,
    concurrency=12,
    hide_blocks=False,
    output_file="output_tutorial_5.csv",
)

## 4. Run Inference

In [12]:
sample = [{'report': sample_report, 'report_id': report_id} for sample_report, report_id in zip(reports, report_ids)]

In [13]:
engine(sample)

Processing items:   0%|          | 0/3 [00:00<?, ?it/s]

Processing items: 100%|██████████| 3/3 [00:09<00:00,  3.01s/it]


In [14]:
results = pd.read_csv("output_tutorial_5.csv", index_col=0)
results

,Pulmonary Embolism_response,Laterality_response,Acuity_response,report,report_id
index,,,,,
0,<answer>\n<Present</Present>\n<initial_answer>...,<answer>\n<initial_answer>\nBilateral\n</initi...,<answer>\n<initial_answer>\nAcute\n</initial_a...,Clinical Information:\n67-year-old male with s...,sample_report_1.txt
1,<answer>\n<Present</Present>\n_initial_answer_...,<answer>\n<initial_answer>\nBilateral</initial...,<answer>\n<initial_answer>\nAcute</initial_ans...,Clinical Information:\n72-year-old female with...,sample_report_2.txt
2,<answer>\n<Present>\n<initial_answer>\nBased o...,<answer>\n<initial_answer>\nBased on the repor...,<answer>\n<initial_answer>\nBased on the repor...,Here is an example radiology report describing...,sample_report_3.txt


In [15]:
engine.save_log("log_tutorial_5.log")